# Save Player Cumulative Season BoxScores 

In [ ]:
import numpy as np
import pandas as pd

from tqdm import tqdm
import time
from nba_api.stats.endpoints import leaguedashplayerstats, leaguegamelog

box_DIR = "../../data/box/"
shiny_DIR = "../../data/shiny/"

In [ ]:
season_start = 1996
season_end = 2024
seasons = np.arange(season_start,season_end,1).astype(str)

league = "NBA"

# Cumulative Boxscores

In [ ]:
def get_box_p_cum(seasons, measure="Base", n=32):
    if measure == "Advanced":
        per_mode = "Per100Possessions"
    else:
        per_mode = "PerGame"
    for season in tqdm(seasons):
        # try:
            season_str = season + "-" + str(int(season) + 1)[-2:]
            stats = leaguedashplayerstats.LeagueDashPlayerStats(
                measure_type_detailed_defense=measure,
                per_mode_detailed=per_mode,
                season=season_str,
            )
            df1 = stats.get_data_frames()[0]
            df = df1.iloc[:, :n]
            if measure == "Advanced":
                measure1 = "Adv"
            else:
                 measure1 = measure
            df.to_parquet(box_DIR + "NBA_Box_P_Cum_" + measure1 + "_" + season + ".parquet")
            time.sleep(0.6)
        # except Exception as error:
        #     continue

In [ ]:
def update_box_p_cum(seasons):
    print("Base")
    get_box_p_cum(seasons, measure="Base", n=32)
    # print("Advanced")
    # get_box_p_cum(seasons, measure="Advanced", n=43)
    # print("Misc")
    # get_box_p_cum(seasons, measure="Misc", n=23)
    # print("Scoring")
    # get_box_p_cum(seasons, measure="Scoring", n=29)

In [ ]:
update_box_p_cum(seasons)

# Individual Game BoxScores

In [ ]:
def update_box_base_p(seasons):
    for season in tqdm(seasons):
        try:
            stats = leaguegamelog.LeagueGameLog(
                player_or_team_abbreviation="P",
                season=season,
                season_type_all_star="Regular Season",
            )
            df = stats.get_data_frames()[0]
            df.to_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + season + ".parquet")
            time.sleep(0.6)
        except Exception as error:
            continue

In [ ]:
update_box_base_p(seasons)

In [ ]:
def update_box_base_p_w(seasons):
    for season in tqdm(seasons):
        try:
            stats = leaguegamelog.LeagueGameLog(
                league_id = "10",
                player_or_team_abbreviation="P",
                season=season,
                season_type_all_star="Regular Season",
            )
            df = stats.get_data_frames()[0]
            df.to_parquet(box_DIR + "WNBA_Box_P_" + "Base" + "_" + season + ".parquet")
            time.sleep(0.6)
        except Exception as error:
            continue

In [ ]:
update_box_base_p_w(seasons)

In [ ]:
dfa = []
for year in range(1946,2024):
    df1 = pd.read_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + str(year) + ".parquet")
    df1[df1["WL"] == 0]["WL"] = "L"
    df1["season"] = year+1
    df1 = df1.fillna(0)
    dfa.append(df1)
df = pd.concat(dfa)
df.loc[df[df["WL"] == 0].index,"WL"] = "L"
df.to_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + "All" + ".parquet")
df.to_parquet(shiny_DIR + "NBA_Box_P_" + "Base" + "_" + "All" + ".parquet")

In [ ]:
dfa = []
for year in range(1997,2024):
    df1 = pd.read_parquet(box_DIR + "WNBA_Box_P_" + "Base" + "_" + str(year) + ".parquet")
    df1[df1["WL"] == 0]["WL"] = "L"
    df1["season"] = year+1
    df1 = df1.fillna(0)
    dfa.append(df1)
df = pd.concat(dfa)
df.loc[df[df["WL"] == 0].index,"WL"] = "L"
df.to_parquet(box_DIR + "WNBA_Box_P_" + "Base" + "_" + "All" + ".parquet")
df.to_parquet(shiny_DIR + "WNBA_Box_P_" + "Base" + "_" + "All" + ".parquet")

In [ ]:
df = pd.read_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + "All" + ".parquet")
df.columns = map(str.lower, df.columns)

In [ ]:
df = pd.read_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + "All" + ".parquet")
df1 = df[~df.duplicated(keep="last")].reset_index(drop=True)
df.to_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + "All" + ".parquet")

In [ ]:
df.to_parquet(shiny_DIR + "NBA_Box_P_" + "Base" + "_" + "All" + ".parquet")

In [ ]:
pd.options.mode.chained_assignment =  None

In [ ]:
cols_i = ['GP', 'W', 'L', 'MIN', 'FGM', 'FGA', 'FG_PCT', 
        'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT','OREB', 'DREB', 'REB', 
        'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS','PLUS_MINUS']

In [ ]:
dfa = []
for year in tqdm(range(1946,2024)):
    df1 = pd.read_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + str(year) + ".parquet")
    df1 = df1.fillna(0)
    df1[df1["WL"] == 0]["WL"] = "L"
    df1g = df1.groupby(["PLAYER_ID","PLAYER_NAME","TEAM_ID","TEAM_NAME"])
    keys = list(df1g.groups)
    dfb = []
    for key in keys:
        dfi = df1g.get_group(key)
        dfi["GP"] = 1
        dfi["W"] = np.where(dfi["WL"] == "W",1,0)
        dfi["L"] = np.where(dfi["WL"] == "L",1,0)
        dfi1 = dfi[cols_i]
        dfii = pd.DataFrame(dfi1.sum()).T
        dfii.iloc[:,3:] = dfii.iloc[:,3:]/dfii["GP"][0]
        dfii.iloc[:,3:] = dfii.iloc[:,3:].round(2)
        dfii["FG_PCT"] = round(dfii["FGM"]/dfii["FGA"],3)
        dfii["FG3_PCT"] = round(dfii["FG3M"]/dfii["FG3A"],3)
        dfii["FT_PCT"] = round(dfii["FTM"]/dfii["FTA"],3)
        dfii[["PLAYER_ID", "PLAYER_NAME", "TEAM_ID", "TEAM_NAME"]] = list(key)
        dfb.append(dfii)
    dfb = pd.concat(dfb)
    dfb["season"] = year + 1
    dfa.append(dfb)
dfa = pd.concat(dfa)
dfs = dfa.replace([np.inf, -np.inf], np.nan)
dfs = dfs.fillna(0).reset_index(drop=True)
dfs.to_parquet(shiny_DIR+"NBA_Box_P_Cum_Base_All.parquet")

In [ ]:
dfa = []
for year in tqdm(range(1997,2024)):
    df1 = pd.read_parquet(box_DIR + "WNBA_Box_P_" + "Base" + "_" + str(year) + ".parquet")
    df1 = df1.fillna(0)
    df1[df1["WL"] == 0]["WL"] = "L"
    df1g = df1.groupby(["PLAYER_ID","PLAYER_NAME","TEAM_ID","TEAM_NAME"])
    keys = list(df1g.groups)
    dfb = []
    for key in keys:
        dfi = df1g.get_group(key)
        dfi["GP"] = 1
        dfi["W"] = np.where(dfi["WL"] == "W",1,0)
        dfi["L"] = np.where(dfi["WL"] == "L",1,0)
        dfi1 = dfi[cols_i]
        dfii = pd.DataFrame(dfi1.sum()).T
        dfii.iloc[:,3:] = dfii.iloc[:,3:]/dfii["GP"][0]
        dfii.iloc[:,3:] = dfii.iloc[:,3:].round(2)
        dfii["FG_PCT"] = round(dfii["FGM"]/dfii["FGA"],3)
        dfii["FG3_PCT"] = round(dfii["FG3M"]/dfii["FG3A"],3)
        dfii["FT_PCT"] = round(dfii["FTM"]/dfii["FTA"],3)
        dfii[["PLAYER_ID", "PLAYER_NAME", "TEAM_ID", "TEAM_NAME"]] = list(key)
        dfb.append(dfii)
    dfb = pd.concat(dfb)
    dfb["season"] = year + 1
    dfa.append(dfb)
dfa = pd.concat(dfa)
dfs = dfa.replace([np.inf, -np.inf], np.nan)
dfs = dfs.fillna(0).reset_index(drop=True)
dfs.to_parquet(shiny_DIR+"WNBA_Box_P_Cum_Base_All.parquet")

In [ ]:
dfs.query("PLAYER_NAME == 'LeBron James'")

In [ ]:
from nba_api.stats.endpoints import leagueleaders

In [ ]:
def get_season_leaders(year):
    season_str = str(year) + "-" + str(year+1)[-2:]
    stats = leagueleaders.LeagueLeaders(
        league_id="00",
        per_mode48="PerGame",
        scope="RS",
        season=season_str,
        season_type_all_star="Regular Season",
        stat_category_abbreviation="PTS"
    )
    df = stats.get_data_frames()[0]
    df.to_parquet(box_DIR+ f"NBA_BOX_P_Lead_Base_{year}.parquet")

In [ ]:
for year in range(1946,2024):
    get_season_leaders(year)
    time.sleep(0.6)

In [ ]:
year = 2017
df1 = pd.read_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + str(year) + ".parquet")
len(df1["PLAYER_ID"].unique())